# VGG

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
IMAGE_SIZE = [224, 224]

In [6]:
train_path = 'F:\Computer Vision\Computer Vision Notebooks\Site Notebooks\Alexnet & Vgg\data\train'
test_path = 'F:\Computer Vision\Computer Vision Notebooks\Site Notebooks\Alexnet & Vgg\data\test'

In [21]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 214s 4us/step


In [22]:
for layer in vgg.layers:
    layer.trainable = False

In [23]:
folders = glob('F:/Computer Vision/Computer Vision Notebooks/Site Notebooks/Alexnet & Vgg/data/train/*')
print(len(folders))

4


In [24]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [25]:
from tensorflow.keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
             optimizer=sgd,
             metrics=['accuracy'])

C:\Anaconda\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [26]:
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range = 0.2,
        zoom_range=0.2,
        horizontal_flip = True,
        fill_mode = 'nearest')

In [27]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [30]:
train_set = train_datagen.flow_from_directory('F:/Computer Vision/Computer Vision Notebooks/Site Notebooks/Alexnet & Vgg/data/train',
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 475 images belonging to 4 classes.


In [31]:
test_set = test_datagen.flow_from_directory('F:/Computer Vision/Computer Vision Notebooks/Site Notebooks/Alexnet & Vgg/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 264 images belonging to 4 classes.


In [37]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1),
                              cooldown=0,
                              patience=5,
                              min_lr = 0.5e-6)

checkpoint = ModelCheckpoint(filepath='mymodel.h5',
                            verbose=1,
                            save_best_only=True)

callbacks = [checkpoint, lr_reducer]

start=datetime.now()

model.fit_generator(
    train_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=5,
    validation_steps=32,
    callbacks = callbacks, verbose=0)

duration = datetime.now() - start
print("Training completed in time: ", duration)

C:\Users\AMMARA~1\AppData\Local\Temp/ipykernel_21352/2705798494.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(



Epoch 1: val_loss improved from inf to 10.49568, saving model to mymodel.h5
Training completed in time:  0:01:47.326168
